In [91]:
class Token:       
    def __init__(self):
        self.balances = {}
        self.totalSupply = 0
    
    def mint(self, address, amount):
        assert(amount > 0)
        self.balances[address] = self.balances.get(address, 0) + amount
        self.totalSupply += amount
    
    def balanceOf(self, address):
        return self.balances.get(address, 0)
    
    def transferFrom(self, sender, recipient, amount):
        assert(amount >= 0)
        if self.balanceOf(sender) >= amount:
            self.balances[sender] = self.balances.get(sender, 0) - amount
            self.balances[recipient] = self.balances.get(recipient, 0) + amount    
            return True
        else:
            return False

In [92]:
token = Token()
token.mint("A", 5)
assert(token.balanceOf("A") == 5)

token.transferFrom("A", "B", 3)
assert(token.balanceOf("B") == 3)
assert(token.balanceOf("A") == 2)

token.transferFrom("A", "C", 2)
assert(token.balanceOf("A") == 0)
assert(token.balanceOf("C") == 2)

In [93]:
import time 
import decimal
from decimal import Decimal 
math = decimal.Context()

# class InterestToken
#  __init__(rate, _rateAccumulator)
#  balanceOfUnderlying(address)
#  accrueInterest()
#  updateRate(update)
#  yearlyRate()

class blockchain:
    
    def __init__(self, _timestamp, _block):
        self.timestamp = _timestamp
        self.block = _block
    
    def incrementBlock(self):
        self.block += 1
        self.timestamp += 15
    
    def updateTimestamp(self, _timestamp):
        self.timestamp = _timestamp
    
    def now(self):
        return self.timestamp


class InterestToken(Token):
    # rate is a yearly rate, converted to a rate compounded per second
    def __init__(self, _blockchain, rate, _rateAccumulator):
        Token.__init__(self)
        self.blockchain = _blockchain
        self.lastUpdate = self.blockchain.now()
        self.rateAccumulator = _rateAccumulator
        # 31622400 is the number of seconds in a year
        self.rate = math.power(1 + Decimal(rate), 1/Decimal(31622400))  
        
    def getCurrentTargetExchangeRate(self):
        return self.rateAccumulator
        
    def accrueInterest(self):
        now = self.blockchain.now()
        if now > self.lastUpdate:
            total_time =  now - self.lastUpdate
            self.rateAccumulator = math.power(self.rate, total_time) * self.rateAccumulator
            self.lastUpdate = now
    
    def updateRate(self, update):
        newRate = math.power(1 + Decimal(update), 1/Decimal(31622400)) - 1
        self.rate = self.rate + newRate
        if self.rate < Decimal(1):
            self.rate = Decimal(1)
    
    def yearlyRate(self):
        return math.power(self.rate, Decimal(31622400)) - 1

In [94]:
chain = blockchain(0,1)
token = InterestToken(chain, .02, 1)
token.mint("A", 1000)
#token.lastUpdate = token.lastUpdate - 31622400
# Note that the per second rates are multiplied, not added
token.updateRate(.001)
token.updateRate(.001)
token.updateRate(-.001)
token.updateRate(.001)
token.updateRate(.001)
#print(token.yearlyRate())
chain.updateTimestamp(31622400)
token.accrueInterest()
assert(int(token.balanceOf("A") * token.getCurrentTargetExchangeRate()) == 1023)

In [95]:
chain = blockchain(1,1)
uToken = InterestToken(chain, .04, 1)
for x in range(10000):
    chain.incrementBlock()
    uToken.accrueInterest()

uToken.rateAccumulator


Decimal('1.000186059700104401460770319')

In [103]:
import string

# class Queueball: 
#   __init__(self, _uToken : Token , _sToken : Token)
#   sell_underlying_for_synthetic(self, address, amount)
#   sell_synthetic_for_underlying(self, address, amount)
#   withdraw_underlying(self, address)

class Queueball:
    def __init__(self, _uToken : Token , _sToken : Token):
        self.uToken = _uToken
        self.pyToken = _sToken
        self.underlying = 0
        self.synthetic  = 0
        self.address = "queueball"
        self.book = dict.fromkeys(list(string.ascii_uppercase), []) 
    
    def sell_underlying_for_synthetic(self, sender, amount):
        uToken.transferFrom(sender, self.address, amount)
        self.underlying += amount
        # inflates supply to create new pyTokens
        if self.underlying > self.synthetic:
            sToken.mint( self.address, self.underlying - self.synthetic)
        amount_scaled_to_pyToken = amount / sToken.getCurrentTargetExchangeRate()
        sToken.transferFrom(self.address, sender, amount_scaled_to_pyToken)
        
    def sell_synthetic_for_underlying(self, sender, amount):
        sToken.transferFrom(sender, self.address, amount)
        amount_scaled_to_underlying = amount * sToken.getCurrentTargetExchangeRate()
        self.synthetic += amount_scaled_to_underlying
        if self.synthetic > self.underlying:
            self.book[sender].append({'amount': amount_scaled_to_underlying, 'height': self.synthetic})
        else:
            uToken.transferFrom(self.address, sender, amount_scaled_to_underlying)

        
    def withdraw_underlying(self, sender):
        for order in self.book[sender]:
            if self.underlying >= order['height']:
                uToken.transferFrom(self.address, sender, order['amount'])
                self.book[sender].remove(order)
    

In [104]:
# Test out Queueball
chain = blockchain(1,1)
uToken = InterestToken(chain, 0,1)
sToken = InterestToken(chain, 0,1)
queue = Queueball(uToken, sToken)
uToken.mint("A", 1000)
sToken.mint("B", 1000)

# Test 1
queue.sell_underlying_for_synthetic("A", 100)
queue.sell_synthetic_for_underlying("B", 100)
queue.withdraw_underlying("A") # should not do anything
queue.withdraw_underlying("B") # should not do anything
assert(uToken.balanceOf("A") ==  900)
assert(uToken.balanceOf("B") ==  100)
assert(sToken.balanceOf("A") ==  100)
assert(sToken.balanceOf("B") ==  900)

#Test 2
queue.sell_underlying_for_synthetic("A", 100)
queue.sell_synthetic_for_underlying("B", 150)
queue.withdraw_underlying("A") # should not do anything
queue.withdraw_underlying("B") # should not do anything
assert(uToken.balanceOf("A") ==  800)
assert(uToken.balanceOf("B") ==  100)
assert(sToken.balanceOf("A") ==  200)
assert(sToken.balanceOf("B") ==  750)

#Test 3
queue.sell_underlying_for_synthetic("A", 100)
queue.withdraw_underlying("B")
queue.withdraw_underlying("A") 

assert(uToken.balanceOf("A") ==  700)
assert(uToken.balanceOf("B") ==  250)
assert(sToken.balanceOf("A") ==  300)
assert(sToken.balanceOf("B") ==  750)

In [124]:
# Test out Queueball
chain = blockchain(1,1)
uToken = InterestToken(chain, 0,1)
sToken = InterestToken(chain, 0.04,1)
queue = Queueball(uToken, sToken)
uToken.mint("A", 1000)
sToken.mint("B", 1000)

#Test 2
queue.sell_synthetic_for_underlying("B", 150)
timestamp = chain.now()
chain.updateTimestamp( timestamp + 60*60*24*30*360)
sToken.accrueInterest()
queue.sell_underlying_for_synthetic("A", 150)
queue.withdraw_underlying("B")
assert(uToken.balanceOf("B") ==  150)
assert(sToken.balanceOf("B") ==  850)

assert(uToken.balanceOf("A") ==  850)
assert(sToken.balanceOf("A") == 150 / sToken.getCurrentTargetExchangeRate() )
queue.sell_underlying_for_synthetic("A", 850)
queue.sell_synthetic_for_underlying("B", 50)
print(sToken.getCurrentTargetExchangeRate())
assert(uToken.balanceOf("B") ==  150 + 50 * sToken.getCurrentTargetExchangeRate())


3.181435529421500642797350854


In [125]:
# Further Test out Queueball
chain = blockchain(1,1)
uToken = InterestToken(chain, 0,1)
sToken = InterestToken(chain, 0,1)
queue = Queueball(uToken, sToken)
uToken.mint("C", 1000)
uToken.mint("D", 1000)
uToken.mint("E", 1000)
sToken.mint("C", 1000)
sToken.mint("D", 1000)
sToken.mint("E", 1000)
# You can permute the following in any order and the tests should still pass

queue.sell_underlying_for_synthetic("C", 100)
queue.sell_underlying_for_synthetic("E", 200)
queue.sell_underlying_for_synthetic("D", 200)
queue.sell_synthetic_for_underlying("D", 300)
queue.sell_underlying_for_synthetic("E", 300)
queue.sell_synthetic_for_underlying("E", 100)
queue.sell_synthetic_for_underlying("E", 400)

#######
queue.withdraw_underlying("C")
queue.withdraw_underlying("D")
queue.withdraw_underlying("E")
assert(uToken.balanceOf("C") ==  900)
assert(uToken.balanceOf("D") ==  1100)
assert(uToken.balanceOf("E") ==  1000)
assert(sToken.balanceOf("C") ==  1100)
assert(sToken.balanceOf("D") ==  900)
assert(sToken.balanceOf("E") ==  1000)

## Simulate

1.00017554555

Decimal('1.000166390173918467456588861')

Decimal('1.000186059700104401460770319')

Decimal('0.033727501488361251126852937')

True

Decimal('5')

1